In [1]:
from joblib import dump, load
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import sisl

def plot_hamiltonian_matplotlib(true_matrix, predicted_matrix, matrix_label=None, figure_title=None, predicted_matrix_text=None, filepath=None, force_max_colorbar_abs_error=None):
    """Matplotlib visualization of error matrices."""

    # Error matrices
    absolute_error_matrix = true_matrix - predicted_matrix
    threshold = 0.001
    mask = true_matrix >= threshold
    relative_error_matrix = np.where(mask, absolute_error_matrix / (true_matrix + threshold) * 100, 0)

    # Colorbar limits
    vmin = np.min([np.min(true_matrix), np.min(predicted_matrix)])
    vmax = np.max([np.max(true_matrix), np.max(predicted_matrix)])
    lim_data = max(np.abs(vmin), np.abs(vmax))

    if force_max_colorbar_abs_error is None:
        lim_abs = np.max(np.abs(absolute_error_matrix))
    else:
        lim_abs = force_max_colorbar_abs_error

    lim_rel = 100.0  # %

    cbar_limits = [lim_data, lim_data, lim_abs, lim_rel]

    # Titles
    if matrix_label is None:
        matrix_label = ''
    titles = [
        "True " + matrix_label,
        "Predicted " + matrix_label,
        "Absolute error (T-P)",
        f"Relative error (T-P)/(T) (masked where T is above {threshold})"
    ]
    cbar_titles = ["eV", "eV", "eV", "%"]

    # Matrices to plot
    matrices = [true_matrix, predicted_matrix, absolute_error_matrix, relative_error_matrix]

    fig, axes = plt.subplots(4, 1, figsize=(10, 16), constrained_layout=True, gridspec_kw={'hspace': 0.15})
    fig.suptitle(figure_title if figure_title else "Matrix Comparison and Errors", fontsize=16)

    for i, (matrix, ax) in enumerate(zip(matrices, axes)):
        im = ax.imshow(matrix, cmap='RdBu', vmin=-cbar_limits[i], vmax=cbar_limits[i])
        ax.set_title(titles[i])
        ax.set_ylabel("Row")
        cbar = fig.colorbar(im, ax=ax, orientation='vertical', fraction=0.045, pad=0.02)
        cbar.set_label(cbar_titles[i])
        ax.set_xlabel("Col")
        ax.set_aspect('auto')

    # Text under predicted matrix (subplot 2)
    if predicted_matrix_text is not None:
        axes[1].text(1.0, -0.25, predicted_matrix_text, ha='right', va='center', fontsize=10, transform=axes[1].transAxes)

    # Absolute error stats (exclude zeros for stats)
    abs_err_nonzero = np.abs(absolute_error_matrix)[absolute_error_matrix != 0]
    if abs_err_nonzero.size > 0:
        mean_abs = np.mean(abs_err_nonzero)
        std_abs = np.std(abs_err_nonzero)
    else:
        mean_abs = std_abs = 0.0
    max_absolute_error = np.max(absolute_error_matrix)
    min_absolute_error = np.min(absolute_error_matrix)
    max_abs = np.max(np.abs([max_absolute_error, min_absolute_error]))

    fig.text(
        0.5, 0.245,
        f"mean_nnz(|T-P|) = {mean_abs:.3f} eV, std_nnz(|T-P|) = {std_abs:.3f} eV, |max| = {max_abs:.3f} eV",
        ha='center', va='center', fontsize=11,
    )

    # Relative error stats (exclude zeros for stats)
    rel_err_nonzero = np.abs(relative_error_matrix)[relative_error_matrix != 0]
    if rel_err_nonzero.size > 0:
        mean_rel = np.mean(rel_err_nonzero)
        std_rel = np.std(rel_err_nonzero)
    else:
        mean_rel = std_rel = 0.0
    max_relative_error = np.max(relative_error_matrix)
    min_relative_error = np.min(relative_error_matrix)
    max_abs_rel = np.max(np.abs([max_relative_error, min_relative_error]))

    fig.text(
        0.5, 0.001,
        f"mean_nnz(|T-P|) = {mean_rel:.3f} %, std_nnz(|T-P|) = {std_rel:.3f} %, |max| = {max_abs_rel:.3f} %",
        ha='center', va='center', fontsize=11,
    )
    axes[-1].set_xlabel(" ")

    # Output
    if filepath:
        plt.savefig(filepath, dpi=150, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show()
        plt.close(fig)

In [ ]:
# 1. Plot structure
# path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/0953-8dcf-411e-ae7b-00f794981cc5") # B-N Overlapped
# path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/c0e0-7549-496d-82e7-c3f6b1d81e58") # B-N No overlapped

# Boron: Grey
# path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/2e65-1feb-4df2-8836-e5513b9bade0") # B-B Overlapped
# path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/7e20-9cdf-4b2c-8134-6cadc8f64c34") # B-B No overlapped
# path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/a4f7-5d3f-4e50-9eda-87df3bf9ba58") # B-B No overlapped (Pero sí en la siguiente cell)

# path = Path("../dataset/SHARE_OUTPUTS_8_ATOMS/39cf-a27b-42dd-a62e-62556132a798") #Hexagonal
# path = Path("../dataset/SHARE_OUTPUTS_8_ATOMS/11ad-ba95-4a26-8f92-5267f5787553") #Cubic

#Val:
path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/fc1c-6ab6-4c0e-921e-99710e6fe41b") #N-N Small overlapped
path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/f016-fb0e-4d5c-b584-4f8987eb5676") #N-N No overlapped

path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/a4e4-2f64-4e68-a37a-9e84eb767a0c") # B-B No overlapped
path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/7bbb-6d51-41eb-9de4-329298202ebf") # B-B overlapped

path = Path("../dataset/SHARE_OUTPUTS_8_ATOMS/173e-fad7-4f78-8350-6759a5471596") #Cubic
path = Path("../dataset/SHARE_OUTPUTS_8_ATOMS/4b9b-20df-4fe5-a669-88ff91902e97") #Hexagonal

# Report:
path = Path("../dataset/SHARE_OUTPUTS_8_ATOMS/0b9e-8905-4d8f-aee8-92b8c892c484") # 8 atms cubic
path = Path("../dataset/SHARE_OUTPUTS_8_ATOMS/82b3-4886-4447-b723-43a3ec6aa15c") #8 atm hBN physical

path = Path("../dataset/SHARE_OUTPUTS_64_ATOMS/5dfc-7444-43ac-a2b0-db75f49a7bb9")
path = Path("../dataset/SHARE_OUTPUTS_64_ATOMS/342f-eb3b-4268-820f-747c3936102b")
path = Path("../dataset/SHARE_OUTPUTS_64_ATOMS/16d6-8097-4bb8-9819-38b7f0f4dfad")

# Report:
path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/d249-97ce-4fdf-8948-cfce4078c8ac") # Training B-N overlap
path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/0a4c-6759-46e6-bf5e-6439eefcaad2") # Val B-N no overlap
path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/abf6-ddbd-4c06-b266-c1d188d0f599") # Training B-B nears
path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/f7bc-fcea-4f67-9e15-4bd1ef5678df") # Val B-B overlap --
path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/e486-4f49-4560-a4d2-0363fd9b5157") # Training N-N overlap
path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/2aa6-e839-4ca4-9e63-0cc27be5f76f") # Val N-N Overlap
path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/18a7-265b-4fbf-aae4-52fb83b4760f"), # Val N-N Overlap AMBOS 2-8 Y 2-8-64

path = Path("../dataset/SHARE_OUTPUTS_8_ATOMS/e1df-2940-4ada-b9c0-d210a6bb2a19") # Training cubic BN
path = Path("../dataset/SHARE_OUTPUTS_8_ATOMS/4ed6-914e-4aa3-923a-53c873f0cc31") # Val cubic BN 5e9e-df50-4bd8-861a-b544fa2116fe
path = Path("../dataset/SHARE_OUTPUTS_8_ATOMS/2800-0707-4395-86fa-85a2b8ed818b") # Training non physical hBN
path = Path("../dataset/SHARE_OUTPUTS_8_ATOMS/33ef-ca33-4bf0-a36e-6e852deab7b5") # VAl non physical hBN --
path = Path("../dataset/SHARE_OUTPUTS_8_ATOMS/d4f5-6b48-494f-b1de-c7e944c09f38") # Training physical hBN
path = Path("../dataset/SHARE_OUTPUTS_8_ATOMS/5721-4d12-41f2-9f31-f7ee4fcf1096") # Val physical hBN

path = Path("../dataset/SHARE_OUTPUTS_64_ATOMS/99a9-5416-41e9-940d-70653c6897f9") # Training 64 of 2-8-64 and 64
path = Path("../dataset/SHARE_OUTPUTS_64_ATOMS/787c-11ce-4307-b603-b6c431698245") # Training 64 of 2-8-64 and 64
path = Path("../dataset/SHARE_OUTPUTS_64_ATOMS/806e-3a78-4e32-9aeb-35dc6e0df137")  # Val 64 of 2-8-64 and 64
path = Path("../dataset/SHARE_OUTPUTS_64_ATOMS/90b9-0d95-49d2-87ed-bd09def740fa") # Val 64 of 2-8-64
path = Path("../dataset/SHARE_OUTPUTS_64_ATOMS/13df-16c2-48b8-aa26-9b0b4b906db2") # Val 64 of 64


path = Path("../dataset/SHARE_OUTPUTS_2_ATOMS/18a7-265b-4fbf-aae4-52fb83b4760f")



print("Plotting structure...")
file = sisl.get_sile(path / "aiida.fdf")
fig = file.plot.geometry(axes="xyz")
# filepath = savedir_struct / f"{n_atoms}atm_{structure}.png"
# fig.write_image(str(filepath))
fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    showlegend=False,
)
fig.show()
file.read_geometry().atoms.Z

Plotting structure...


array([5, 5], dtype=int32)

In [3]:
# 2. Plot hamiltonian
from tools.plot import plot_error_matrices_big


calculations_path = Path('../scripts/debug_data.joblib')

data = load(calculations_path)

print(f"Loading data from {calculations_path}")
try:
    data = load(calculations_path)
    print("Results loaded!")
except FileNotFoundError:
    raise FileNotFoundError(f"Could not find the saved calculations at {calculations_path}")

# Reconstruct your tuples and labels
train_data   = (data['train_true'],   data['train_pred'])
val_data     = (data['val_true'],     data['val_pred'])
train_labels = data['train_labels']
val_labels   = data['val_labels']


# Unpack for clarity
train_true, train_pred = train_data
val_true,   val_pred   = val_data

n_train_samples = len(train_data[0])
n_val_samples = len(val_data[0])

print("Plotting hamiltonian...")
h_true = train_true[0]
h_pred = train_pred[0]
title = f"test"
# filepath = savedir / f"{n_atoms}atm_{structure}_hamiltonian.png"
plot_error_matrices_big(true_matrix=h_true.todense(), predicted_matrix=h_pred.todense())


ModuleNotFoundError: No module named 'tools'